In [2]:
import cv2
import skimage
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
folder = r"C:\Users\benjm\Downloads\Workspace\final"       # run for both test and train
mask_folder = r"C:\Users\benjm\Downloads\Workspace\Masks"
mask_ending = ['_occluded_root_mask.tif', '_root_mask.tif', '_shoot_mask.tif', '_seed_mask.tif']
measurement = True

for image in os.listdir(folder):
    image_path = os.path.join(folder, image)
    im = cv2.imread(image_path, 0)
    if not(im is None):
        im = cv2.medianBlur(im, 5)
        th, output_im = cv2.threshold(im, 10, 255, cv2.THRESH_BINARY)
        output_im = cv2.dilate(output_im, np.ones((3, 3), np.uint8), iterations=3)
        output_im = cv2.erode(output_im, np.ones((3, 3), np.uint8), iterations=10)

        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(output_im)

        largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])

        x,y,w,h,_ = stats[largest_label]
        top_left = (x,y)
        bottom_right = (x+w,y+h)
        
        im = im[y:y+h, x:x+w]
        output_path = os.path.join(folder + r"\cut", image)
        im = cv2.imwrite(output_path, im)
        if measurement:
            continue
        for mask in mask_ending:
            name = image.split(".")[0] + mask
            mask_path = os.path.join(mask_folder, name)
            mask_im = cv2.imread(mask_path, 0)
            if not(mask_im is None):
                mask_im = mask_im[y:y+h, x:x+w]
                output_path = os.path.join(mask_folder + r"\cut", name)
                mask_im = cv2.imwrite(output_path, mask_im)
            else:
                print(mask_path)
    else:
        print(image_path)
    

C:\Users\benjm\Downloads\Workspace\final\cut


In [3]:
print(len(os.listdir(r"C:\Users\benjm\Downloads\Workspace\train")) + len(os.listdir(r"C:\Users\benjm\Downloads\Workspace\test")) -2 == len(os.listdir(r"C:\Users\benjm\Downloads\Workspace\train\cut")) + len(os.listdir(r"C:\Users\benjm\Downloads\Workspace\test\cut")))

True
